# Riconoscimento Linee

Ora che abbiamo calibrato la camera possiamo dedicarci al primo e fondamentale passo per un algoritmo di guida autonoma: il **riconoscimento delle linee stradali**.

La città di CheemsCity è caratterizzata da strade nere con linee bianche a segnalarne i bordi e tutte le analisi che andremo a fare adesso si baseranno su questo assunto;  
qual'ora il vostro robot viaggiasse in una città con regole diverse vi basterà cambiare poche cose.

## Matrice Omografica

La **matrice omografica** è un particolare tipo di matrice che ci permette di passare facilmente da un sistema di riferimento ad un altro, ed è molto utile per descrivere rototraslazioni.   
Noi andremo a calcolare la matrice che codifica il passaggio dal sistema di riferimento della foto a quello del mondo reale (da 2D a 3D), in particolare il centro del nuovo sistema di  
riferimento si troverà nella proiezione sul terreno del centro della fotocamera.

Per poter calcolare la matrice vi occorrerà il foglio con la scacchiera che dovreste aver stampato nel punto precedente.

In [ ]:
import cv2
import time
import numpy as np
import yaml
import glob
import matplotlib.pyplot as plt

In [ ]:
from picamera import PiCamera

Andiamo ora ad importare la configurazione che avevamo effettuato nel foglio precedente.

In [ ]:
file = open("part1/FinalCalibration.yml", "r")
calibration_data = yaml.load(file, Loader=yaml.UnsafeLoader)
matrix = calibration_data['camera_matrix']
dist_coef = calibration_data['distortion_coefficient']

Calcoliamo quindi una mappa che permetterà di correggere gli errori della camera in modo veloce.  
**NOTA**: *mappa è spesso usato come sinonimo di funzione.*

In [ ]:
#TO-DO: inserire dimensioni dell'immagine o fare programma che calcola.
mapx, mapy = cv2.initUndistortRectifyMap(
                cam_matrix, dist_coeff, None, cam_matrix, (w, h), 5)

Creiamo ora una funzione che ci permetterà di sfruttare questa mappa per la correzzione delle immagini.

**TIPS**: *l'utilizzo del try and else ci permette di gestire eventuali problemi, è molto consigliato il suo utilizzo in casi in cui l'utente potrebbe involontariamente andare a creare problemi.*

In [ ]:
def undistort_faster(image, mapx, mapy):
    try:
        return cv2.remap(image, mapx, mapy, cv2.INTER_LINEAR)
    else:
        return image

Posizioniamo la scaccchiera a terra e mettiamo la camera nell'apposito spazio. Una volta fatto andiamo a definire delle variabili che ne indicheranno la distanza dal punto zero dello scacchiera.

1. **offsety**: distanza lungo l'asse parallelo al lato più vicino della scacchiera.
2. **offsetx**: distanza camera-scacchiera, asse perpendicolare a questa

In [ ]:
camera_calibration_square_size: 0.018
offsety = 3 * camera_calibration_square_size
offsetx = 0.102 
board_offset = np.array([offsetx, -offsety])
nx = #nx
ny = #ny

E ora come nel caso precedente della calibrazione della camera, sfrutteremo le foto per il calcolo della matrice omogenea. Il vantaggio è che in questo caso ce ne servirà solo una. Il primo blocco è per chi utilizza una picamera, il secondo per quelle USB.

In [ ]:
import picamera
#programma per picamera raspberry pi

camera = Picamera()
print("avvio camera")
camera.start_preview()
print("foto tra 4 secondi")
time.sleep(4)
photo_name = "part2/foto/omografia.jpg"
camera.capture(photo_name)
camera.stop_preview()

In [ ]:
#programma per camera nativa o USB

cam = cv2.VideoCapture(0)
while True:
    print("avvio camera")
    ret, image = cam.read()
	cv2.imshow('photo',image)
	k = cv2.waitKey(1)
    if k == ord(s):
        photo_name = "part2/foto/omografia.jpg"
        cv2.imwrite(photo_name, image)
    if k == ord(q):
		break

Controlliamo che la foto scattata sia valida per la calibrazione, in caso contrario rirunna le celle precedenti.

In [ ]:
image = glob.glob('part2/foto/omografia.jpg')

for fname in image:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (nx,ny), None)
    if ret == True:
        total = total + 1
print("numero di foto buone: {}".format(total))

Definiamo e cerchiamo i punti necessari per la calibrazione, saranno gli stessi del foglio precedente.

**NOTA:** *andiamo ad effetturare un reverse sui punti perchè gli array numpy hanno l'origine in alto a sinistra, mentro noi la vogliamo in basso a sinistra. Questo però ci porterebbe ad avere il punto nella parte alta e sinistra della scacchiera associato con l'ultima coordinata, ottenendo così un'immagine speculare e per questo dovremo andare a moltiplicare la prima riga della matrice omografica per -1.*

In [ ]:
src_pts = []
for r in range(ny):
    for c in range(nx):
        src_pts.append(
            np.array([r * square_size, c * square_size],
                        dtype='float32') + board_offset)

src_pts.reverse()

imgpoints = []

In [ ]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

ret, corners = cv.findChessboardCorners(gray, (nx,ny), None)

if ret == True:
    corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
    imgpoints.append(corners2)

Procediamo a calcolare la matrice grazie alla funzione di opencv e successivamente salviamola in un file yaml.

In [ ]:
H, _mask = cv2.findHomography(
        imgpoints.reshape(len(imgpoints), 2),
        np.array(src_pts), cv2.RANSAC)

#vedere se funziona
H[1,:] = H[1,:] * -1

In [ ]:
calibration_data = {
    calibration_data = {
            "H_matrix": H,
}
with open('part2/Homography.yml', 'w') as outfile:
    yaml.dump(calibration_data, outfile, default_flow_style = False)

## Riconoscimento linee codice ##

Ora che abbiamo la matrice omografica possiamo passare alla parte principale del tutorial; in breve quello che andremo a fare sarà:
1) estrarre tutti i pixel di color bianco dall'immagine.
2) cercare i punti con gradiente più alto nell'immagine.
3) intersecare i due risultati per ottenere un riconosciento ottimale.
4) trovare delle rette nell'immagine risultato.

Partiamo mostrando tutti i passaggi su un'immagine di prova, e poi andremo a definire una pipeline riutilizzabile per processare immagini real-time

In [ ]:
#TO-DO: definire importazione immagine

Andiamo a rettificare l'immagine utilizzando la funzione definita prima (undistort_faster);  
Creeremo poi 2 copie in modo da non andare a modificare direttamente l'immagine di partenza con i filtri.

In [ ]:
image = undistort_faster(image, mapx, mapy)
lane_image = np.copy(image[200:, :, :])
canny_image = np.copy(image[200:, :, :])

Sulla prima immagine copiata andremo ad applicare tutti gli algoritmi legati al filtraggio per colore, mentre nella seconda quelli legati alla ricerca di rette.

TO-DO: parlare della convenzione HSV

In [ ]:
sens = 100
lower_white = np.array([0, 0, 255 - sens])  #convenzione HSV
upper_white = np.array([255, sens, 255])

Procediamo ora ad isolare i pixel di colore bianco, ottenendo così una matrice con le stesse dimensioni dell'immagine e valore True dove presente i pixel bianchi.

In [ ]:
frameHSV = cv2.cvtColor(lane_image, cv2.COLOR_BGR2HSV)
frameHSV = cv2.inRange(frameHSV, lower_white, upper_white)

Per evitare di trovarci con linee spezzate o bucate utilizziamo una trasformazione morfologica basata su un kernel 3x3 chiamata [dilate]('https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html').


In [ ]:
#STEP 3: chiudere eventuali buchi con un dilate:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
frameHSV = cv2.dilate(frameHSV, kernel)

Andiamo ora ad applicare l'algoritmo canny sulla seconda immagine (speigare brevemente canny e mettere link).

In [ ]:
edges = cv2.Canny(canny_image, 80, 200, apertureSize=3)

Uniamo ora i due risultati -> le 2 immagini saranno infatti caratterizzate da 2 colori (bianco e nero, True o False) e sarà quindi possibile eseguire un'operazione di intersezione tra queste.

In [ ]:
edge_color = cv2.bitwise_and(frameHSV, edges)

Cerchiamo ora delle rette nell'immagine attraverso l'algoritmo Hough (mettere spiegazione breve e link). Spiegare anche brevemente parametri.

In [ ]:
lines = cv2.HoughLinesP(edge_color,
                        rho=1,
                        theta=np.pi / 180,
                        threshold=2,
                        minLineLength=3,
                        maxLineGap=1,
                        lines=np.array([]))

To-DO: inserire rappresentazione di queste rette nell'immagine.

Ora abbiamo un insieme di punti legati alle rette trovate dall'algoritmo Hough, questi punti saranno definiti nel sistema di riferimento dell'immagine. Per un ottimo controllo di un eventuale veicolo sarà bene trasformarle nel sistema di riferimento del robot tramite la matrice omografica.

l'algoritmo Hough ritornerà un array nX4, composto quindi da n righe di valori x1,y1,x2,y2. Quello che facciamo è dividere i 4 punti in 2 array, il primo contenente solo i punti 1 (x1,y1), il secondo solo quelli 2. Come ultimo passaggio aggiungiamo il primo array ad una lista contenente tutti i punti 1 e il secondi ad una lista contenente tutti i punti 2.

In [ ]:
if lines is not None:
    for line in lines:
        p1, p2 = line.reshape(2, 2)
        road_points_p1.append(p1)
        road_points_p2.append(p2)

Per poter lavorare più facilmente con i punti trasformiamo le liste in array numpy.

In [ ]:
road_vec1 = np.array(road_points_p1, ndmin=2)
road_vec2 = np.array(road_points_p2, ndmin=2)

Ricordiamo che i punti sono stati calcolati su un'immagine tagliata, per garantire quindi la veridicità della trasformazione dovrò aggiungere l'altezza del pezzo tagliato ai valori dell'altezza dei punti. Questo verrà fatto creando un array (0,h) e sommandolo agli array dei punti.

In [ ]:
roi_h = 200
roi_vect = np.array((0, roi_h))

road_vec1 = (road_vec1 + roi_vect)
road_vec2 = (road_vec2 + roi_vect)

definiamo ora una funzione sky_view_points per il calcolo di questi.

In [ ]:

    def sky_view_points(points, H):
        vector = np.append(points, np.array([1]))
        ground_point = np.dot(H, vector)
        x = ground_point[0]
        y = ground_point[1]
        z = ground_point[2]

        skyPoints = np.array([(y / z), (x / z)])
        return skyPoints

creiamo due nuove liste contenente i valori dei punti ma post trasformazione omografica. Come prima procediamo poi a trasformarla in un array numpy.

In [ ]:
numPoints = road_vec1.shape[0] if road_vec1.shape[1] == 2 else 0
sky_list1 = []
sky_list2 = []
for i in range(numPoints):
    sky_list1.append(sky_view_points(Road_vec1[i, :]))
    sky_list2.append(sky_view_points(Road_vec2[i, :]))

In [ ]:
sky_points1 = np.array(sky_list1, ndmin=2)
sky_points2 = np.array(sky_list2, ndmin=2)